In [3]:
#author: Robert Niemann

In [4]:
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [5]:
# Writes the url string needed based off which stock the user inputs. url leads to the yahoo historical data on that stock.
def obtainURL(stockName):
    stockName = stockName.upper()
    sampleURL = "https://finance.yahoo.com/quote/" + stockName + "/history?p=" + stockName
    return sampleURL

In [6]:
#retreives the html code from the url, extracts the table with relevant information and returns it, still in html form.
def accessWebData(sampleURL):
    #retreives information from site
    yahooPage = urlopen(sampleURL)
    stockSoup = bs4.BeautifulSoup(yahooPage, 'html.parser')
    
    #extracts table from html and returns as a list of rows
    completeTableObject = stockSoup.find_all('div', {'class': 'Pb(10px) Ovx(a) W(100%)'})
    completeTable = completeTableObject[0].find_all('table', {'class': 'W(100%) M(0)'})
    tableBody = completeTable[0].find_all('tbody')[0]
    tableBodyRows = tableBody.find_all('tr', {'class': 'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'})
    return tableBodyRows

In [12]:
#simple method, only removes the commas from the parameter and returns the 'de-comma'd' version
#made to save space in makeTableDF
def corrected(data):
    if (',' in data):
        newData = data.replace(',','')
    else:
        newData = data
    return newData

In [7]:
#takes the rows retreived from the previous method and uses them to create a dataframe to be returned
def makeTableDF(tableBodyRows):
    #dict of dicts where each represents a row/date, containing 6 different data poitns
    bodyRowsDict = {}
    #counting in reverse allows the program to match dates appropriately with the snp500 index. latest row is always 100
    counter = len(tableBodyRows)
    
    for row in tableBodyRows:
        #in some cases, a row will consist of only one line, which the rest of this method can't appropriately parse
        if (len(row) < 3):
            continue
        
        row_data = row.find_all('td')
        
        #taking advantage of mutability/assignment with these two lines, so I can always refer to the current row as 'sampleDict'
        bodyRowsDict[counter] = {}
        sampleDict = bodyRowsDict[counter]
        
        #if the current row is 'valid', one is removed from the counter to align the dates properly
        counter = counter-1
        
        #assigns the appropriate data from html to the dict
        sampleDict['Open'] = corrected(row_data[1].text)
        sampleDict['High'] = corrected(row_data[2].text)
        sampleDict['Low'] = corrected(row_data[3].text)
        sampleDict['Close*'] = corrected(row_data[4].text)
        sampleDict['Adj Close**'] = corrected(row_data[5].text)
        sampleDict['Volume'] = corrected(row_data[6].text)
        
    #returns the data of all rows as a dataframe, obtained from a dict of dicts
    return pd.DataFrame.from_dict(bodyRowsDict, orient='index')

In [8]:
#compilation of the previous methods, only one used in the main program. given the name of a stock, this method returns
#the dataframe obtained from the site's html code. default returns the snp500 data
def scrape(stockName = '%5EGSPC'):
    url = obtainURL(stockName)
    data = accessWebData(url)
    datadf = makeTableDF(data)
    return datadf